In [1]:
import copy
from datetime import datetime
import json
from pathlib import Path
import pandas
import re
import os
import sys

repo_path = str(Path().resolve().parent)
os.chdir(repo_path)
sys.path.append(repo_path)

In [20]:
EMPTY_MEASUREMENT_DICT= {
    "geo_coordinates": {
            "lat": None, "lon": None
        },
    "time": "",
    "notes": "",
    "depth": None,
    "measures": []
}
EMPTY_DATA_DICT = {
        "date": "",
        "estuary_name": "",
        "participants": [],
        "notes": "",
        "spatial_turbidity_measurement": {
            "turbidity_unit": "FNU",
            "depth_unit": "meter",
            "measurements_list": []
        }
    }
LOC_LON_LAT_REGEX = r"(\w+\s\w+)\s\((-?\s?\d+\.\d+), (-?\s?\d+\.\d+)\)"

In [3]:
turbidity_file_path = "data/Turbidité.xlsx"

In [35]:
estuary_name_list = [
    "cocagne",
    "bouctouche",
    "morell",
    "west",
    "dunk"
]

In [36]:
def create_data_dict_from_df(estuary_name, df):
    data_dict = copy.deepcopy(EMPTY_DATA_DICT)
    data_dict["date"] = df["Date (yyyy-mm-dd)"][0].strftime("%Y-%m-%d")
    data_dict["estuary_name"] = estuary_name
    data_dict["participants"] = ["Alexandre Parlee"]
    data_dict["notes"] = df["Conditions"][0]
    measurements_list = data_dict["spatial_turbidity_measurement"]["measurements_list"]
    time = datetime.strptime(df["Time"][0], "%Ih%M %p").strftime("%H:%M")
    for col in df.columns[-4:]:
        measurement_dict = copy.deepcopy(EMPTY_MEASUREMENT_DICT)
        measurement_values = df[col].values
        if all(pandas.isna(measurement_values)) is False:
            loc_lon_lat_match = re.search(LOC_LON_LAT_REGEX, col)
            location = loc_lon_lat_match.group(1)
            latitude = float(loc_lon_lat_match.group(2).replace("- ", "-"))
            longitude  = float(loc_lon_lat_match.group(3).replace("- ", "-"))
            measurement_dict["geo_coordinates"]["lat"] = latitude
            measurement_dict["geo_coordinates"]["lon"] = longitude
            measurement_dict["time"] = time
            measurement_dict["notes"] = f"{df['Turbidity Measurement'][0]} {location}"
            measurement_dict["measures"] = list(measurement_values)

            measurements_list.append(measurement_dict)
    return data_dict

In [45]:
for estuary_name in estuary_name_list:
    data_dict_list = []
    sheet_name = estuary_name.capitalize() + " River"
    df = pandas.read_excel(turbidity_file_path, sheet_name=sheet_name, header=None)
    number_of_mesaurement = (df[0] == "Turbidity Measurement").sum()
    for i in range(number_of_mesaurement):
        col_values = df.iloc[7*i].values
        df_measure = df.iloc[7*i+1:7*(i+1) - 1].reset_index(drop=True)
        df_measure.columns = col_values
        data_dict_list.append(create_data_dict_from_df(estuary_name, df_measure))
    with open(Path("data/field_work/json/summer_2023", f"{estuary_name}.json"), "w") as f:
        json.dump(data_dict_list, f, indent=2)